In [1]:

import torch
import numpy as np
import scipy as sp
import sys
sys.path.append('/Users/judd/Documents/optimisation_of_superconduncting_circuits/core')
from fluxonium import Fluxonium 
import general as general
import scqubits as sc


In [2]:
EJ = torch.rand(1, requires_grad=True, dtype=torch.double)
EC = torch.rand(1, requires_grad=True, dtype=torch.double)
EL = torch.rand(1, requires_grad=True, dtype=torch.double)
flux = torch.tensor([0.5], requires_grad=True, dtype=torch.double)

EJ.data = EJ.data * (20 - 2.5) + 2.5
EC.data = EC.data * (8 - 1e-3) + 1e-3
EL.data = EL.data * (10 - 2e-1) + 2e-1

dim = 110

fluxonium_auto = Fluxonium(EJ, EC, EL, flux, dim, "auto_H")
fluxonium_create = Fluxonium(EJ, EC, EL, flux, dim, "create_H")
fluxonium_sym = Fluxonium(EJ, EC, EL, flux, dim, "sym_H")
fluxonium_sc = sc.Fluxonium(EJ=EJ.item(), EC=EC.item(), EL=EL.item(), flux=flux.item(), cutoff=dim)

##Not Equal

print(general.t2_rate(
    fluxonium_sym, fluxonium_sym.t1_supported_noise_channels(), fluxonium_sym.tphi_supported_noise_channels()
))

print(general.t2_rate(
   fluxonium_auto, fluxonium_auto.t1_supported_noise_channels(), fluxonium_auto.tphi_supported_noise_channels()
))

 /Users/judd/Documents/optimisation_of_superconduncting_circuits/core/general.py: 75

tensor([[0.0049]], dtype=torch.float64, grad_fn=<AddBackward0>)
tensor([[0.7266]], dtype=torch.float64, grad_fn=<AddBackward0>)


In [4]:
#OPTIMISATION

#FIXED PARAMS
dim = 110
learn_rate = 1
flux = torch.tensor([0.5], requires_grad=True, dtype=torch.double)

#VARIABLE PARAMS
EJ = torch.rand(1, requires_grad=True, dtype=torch.double)
EC = torch.rand(1, requires_grad=True, dtype=torch.double)
EL = torch.rand(1, requires_grad=True, dtype=torch.double)
flux = torch.tensor([0.5], requires_grad=True, dtype=torch.double)

EJ.data = EJ.data * (20 - 2.5) + 2.5
EC.data = EC.data * (8 - 1e-3) + 1e-3
EL.data = EL.data * (10 - 2e-1) + 2e-1

EJ_bounds = [2.5, 150]
EC_bounds = [1e-3, 8]
EL_bounds = [2e-1, 10]

#RECORDING VALUES
EJ_vals = [EJ.item()]
EC_vals = [EC.item()]
EL_vals = [EL.item()]
R2_rate = [1e100, 1e99]

# GRADIENT DESCENT
i = 0
while R2_rate[-1] < R2_rate[-2]:
    i+=1

    fluxonium = Fluxonium(EJ, EC, EL, flux, dim, 'create_H')
    R2  = general.t2_rate(fluxonium, fluxonium.t1_supported_noise_channels(), fluxonium.tphi_supported_noise_channels())
    R2_rate.append(R2.item())

    R2.backward()

    with torch.no_grad():
        #If paramater value drops below lower bound, it will remain on the lower bound
        ##WHY DO WE minus the gradient and not add?
        EJ.data = EJ - EJ.grad*learn_rate if EJ - EJ.grad*learn_rate > EJ_bounds[0] else torch.tensor(EJ_bounds[0])
        EL.data = EL - EL.grad*learn_rate if EL - EL.grad*learn_rate > EL_bounds[0] else torch.tensor(EL_bounds[0])
        EC.data = EC - EC.grad*learn_rate if EC - EC.grad*learn_rate > EC_bounds[0] else torch.tensor(EC_bounds[0])
    
    #RECORDING VALUES FOR PLOT
    EJ_vals.append(EJ.item())
    EC_vals.append(EC.item())
    EL_vals.append(EL.item())

    print(f"{i}: R2: {R2.item()}, EJ: {EJ.item()}, EL: {EL.item()}, EC: {EC.item()}")

RuntimeError: linalg_eigh_backward: The eigenvectors in the complex case are specified up to multiplication by e^{i phi}. The specified loss function depends on this quantity, so it is ill-defined.